In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten
from keras.optimizers import Adam 
from keras.preprocessing.image import ImageDataGenerator

# Initializing image data generator with rescaling

In [2]:
train_data_gen = ImageDataGenerator(rescale = 1./255)
validation_data_gen = ImageDataGenerator(rescale = 1./255)

# Preprocess all the train images

In [4]:
train_generator = train_data_gen.flow_from_directory(
                                    
                '/kaggle/input/fer2013/train',
                target_size = (48,48),
                batch_size = 64,
                color_mode = "grayscale",
                class_mode = 'categorical')

Found 28709 images belonging to 7 classes.


# Preprocess all the test images

In [5]:
validation_generator = validation_data_gen.flow_from_directory(
    
                '/kaggle/input/fer2013/test',
                target_size = (48,48),
                batch_size = 64,
                color_mode = "grayscale",
                class_mode = 'categorical')

Found 7178 images belonging to 7 classes.


# Creating the Deep Learning Model

In [9]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32,kernel_size = (3,3),activation = 'relu',input_shape = (48,48,1)))
emotion_model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128,kernel_size = (3,3),activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128,kernel_size = (3,3),activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024,activation = 'relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7,activation = 'softmax'))

# Compiling the Model

In [10]:
emotion_model.compile(loss = 'categorical_crossentropy',optimizer = Adam(lr = 0.0001,decay = 1e-6 ),metrics = ['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


# Training the Model

In [13]:
emotion_model_info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch = 28709//64,
    epochs = 100,
    validation_data = validation_generator,
    validation_steps = 7178//64,
    initial_epoch = 50)

Epoch 51/100


/tmp/ipykernel_28/237661982.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


448/448 [==============================] - 33s 74ms/step - loss: 0.3577 - accuracy: 0.8710 - val_loss: 1.2398 - val_accuracy: 0.6246
Epoch 52/100
448/448 [==============================] - 33s 73ms/step - loss: 0.3519 - accuracy: 0.8738 - val_loss: 1.2378 - val_accuracy: 0.6221
Epoch 53/100
448/448 [==============================] - 37s 83ms/step - loss: 0.3348 - accuracy: 0.8809 - val_loss: 1.2496 - val_accuracy: 0.6222
Epoch 54/100
448/448 [==============================] - 37s 82ms/step - loss: 0.3285 - accuracy: 0.8816 - val_loss: 1.2592 - val_accuracy: 0.6228
Epoch 55/100
448/448 [==============================] - 37s 82ms/step - loss: 0.3175 - accuracy: 0.8878 - val_loss: 1.2965 - val_accuracy: 0.6235
Epoch 56/100
448/448 [==============================] - 37s 83ms/step - loss: 0.3144 - accuracy: 0.8882 - val_loss: 1.2787 - val_accuracy: 0.6193
Epoch 57/100
448/448 [==============================] - 34s 77ms/step - loss: 0.3016 - accuracy: 0.8909 - val_loss: 1.2961 - val_accuracy

# Save the model

In [14]:
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

In [15]:
# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')
